In [24]:
# The following code is coppied and adapted from the following webpage:
# https://www.geeksforgeeks.org/emotion-detection-using-bidirectional-lstm/
# The attention layer is coppied and adapted from the following page:
# https://zhuanlan.zhihu.com/p/29201491?utm_id=0
# https://github.com/stevewyl/nlp_toolkit/tree/master/nlp_toolkit/modules/attentions
# https://github.com/sujitpal/eeap-examples/blob/master/src/custom_attn.py

In [2]:
# Importing the required libraries
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Bidirectional
from nltk.tokenize import word_tokenize,sent_tokenize
from keras.layers import *
from sklearn.model_selection import cross_val_score,train_test_split
import nltk
import pandas as pd
import tensorflow as tf


nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\4K\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv('text_emotion_isear_edit2.csv', header=1)

In [4]:
#Changed a bit as now we have a DataFrame
df.drop(df[df['SIT'] == '[ No response.]'].index, inplace = True)
#let's get rid of anger and disgust to have a try
df = df[~df['EMOTION'].isin(['guilt', 'disgust'])]
print (df['EMOTION'].value_counts() )

number_emotions = df['EMOTION'].unique()

EMOTION
fear       1074
sadness    1068
anger      1062
joy        1061
shame      1060
Name: count, dtype: int64


In [5]:

#make all contexts (sentences) in dataset a list
texts =df["SIT"]

#HIGHTLIGHT
#Convert each text value to lowercase using the .str.lower() method
lower_texts = texts.str.lower()

# Each  sentence in feel_arr is tokenized by the help of work tokenizer.
# If I have a sentence - 'I am happy'. 
# After word tokenizing it will convert into- ['I','am','happy']
lower_texts = [word_tokenize(sent) for sent in lower_texts]

#Now I added some codes to see how many words it has for the longest text
longest_text = max(lower_texts, key=len)
print (longest_text)
print("Sublist with the most elements:", len(longest_text))

['this', 'is', 'an', 'event', 'i', 'will', 'never', 'forget', '.', 'i', 'am', 'considered', 'a', 'good', 'ã¡', 'mimic', '.', 'this', 'particular', 'day', 'we', 'were', 'waiting', 'for', 'a', 'professor', 'ã¡', 'to', 'take', 'her', 'class', '.', 'this', 'professor', 'had', 'a', 'peculiar', 'accent', 'and', 'a', 'ã¡', 'very', 'horrible', 'way', 'of', 'lecturing', 'and', 'to', 'top', 'it', 'all', 'we', 'had', 'this', 'ã¡', 'class', 'at', 'the', 'fag', 'end', 'of', 'the', 'day', '.', 'so', 'this', 'particular', 'day', 'i', 'got', 'ã¡', 'into', 'my', 'element', 'and', 'started', 'imitating', 'the', 'professor', ',', 'and', 'the', 'ã¡', 'professor', 'entered', 'the', 'class', 'right', 'in', 'the', 'middle', 'of', 'it', '.', 'i', 'was', 'ã¡', 'unaware', 'of', 'her', 'presence', ',', 'some', 'of', 'my', 'friends', 'tried', 'to', 'warn', 'me', 'but', 'ã¡', 'it', 'was', 'of', 'no', 'avail', 'as', 'i', 'was', 'too', 'engrossed', 'in', 'mimicing', '.', 'then', 'i', 'ã¡', 'suddenly', 'noticed', 'th

In [6]:

# Defined a function padd in which each sentence length is fixed to 200.
# If length is less than 200 , then the word- '<padd>' is append
def padd(arr):
    for i in range(200-len(arr)):
        arr.append('<pad>')
    return arr[:200]
   
# call the padd function for each sentence in feel_arr
for i in range(len(lower_texts)):
    lower_texts[i]=padd(lower_texts[i])
 

# Glove vector contains a 50 dimensional vector corresponding 
# to each word in dictionary.
vocab_f = 'glove.6B.50d.txt'
 
# embeddings_index is a dictionary which contains the mapping of
# word with its corresponding 50d vector.
embeddings_index = {}
with open(vocab_f, encoding='utf8') as f:
    for line in f:
        # splitting each line of the glove.6B.50d in a list of 
        # items- in which the first element is the word to be embedded,
        # and from second to the end of line contains the 50d vector.
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
         
# Embedding each word of the feel_arr
embedding_texts = []
# Iterate through each sub-list (sentences)
for word_list in lower_texts:
    # Initialize an empty list to store the embedding vectors for each word
    word_vectors = []
    for word in word_list:
        if word in embeddings_index:
            word_vector = embeddings_index[word]
            word_vectors.append(word_vector)
        else:
            # if the word to be embedded is '<padd>' append 0 fifty times
            word_vector = np.zeros(50, dtype='float32')
            word_vectors.append(word_vector)
    embedding_texts.append(word_vectors)


In [7]:
#embeeding_texts cannot be a list, for the website he made it NumPy-array
X = np.array(embedding_texts)

#One-Hot Encoding for all emotions, 
#instead of just give them numbers, by doing so we will have seven outputs
Y = pd.get_dummies(df['EMOTION'], prefix='emotion')

# Split into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.2,random_state=2)

In [23]:
"""
class BiLSTMModel:
    def __init__(self):
        self.model = Sequential()
        self.model.add(Input(shape=(200, 50)))
        
        self.model.add(Bidirectional(LSTM(64, input_shape=(200, 50))))

        self.model.add(Dropout(0.2))
        
        self.model.add(Dense(5, activation='softmax'))
        self.model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
 
    def fit(self, X, Y, epochs, batch_size,verbose):
        self.model.fit(X, Y, epochs=epochs, batch_size=batch_size,verbose=verbose)
 
    def evaluate(self, X, Y, batch_size,verbose):
        return self.model.evaluate(X, Y, batch_size=batch_size,verbose=verbose)
 
    def predict(self, X):
        return self.model.predict(X)
"""


"\nclass BiLSTMModel:\n    def __init__(self):\n        self.model = Sequential()\n        self.model.add(Input(shape=(200, 50)))\n        \n        self.model.add(Bidirectional(LSTM(64, input_shape=(200, 50))))\n\n        self.model.add(Dropout(0.2))\n        \n        self.model.add(Dense(5, activation='softmax'))\n        self.model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])\n \n    def fit(self, X, Y, epochs, batch_size,verbose):\n        self.model.fit(X, Y, epochs=epochs, batch_size=batch_size,verbose=verbose)\n \n    def evaluate(self, X, Y, batch_size,verbose):\n        return self.model.evaluate(X, Y, batch_size=batch_size,verbose=verbose)\n \n    def predict(self, X):\n        return self.model.predict(X)\n"

In [8]:
@keras.saving.register_keras_serializable()
class Attention(Layer):
    def __init__(self, attention_size, **kwargs):
        self.attention_size = attention_size
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        # W: (EMBED_SIZE, ATTENTION_SIZE)
        # b: (ATTENTION_SIZE, 1)
        # u: (ATTENTION_SIZE, 1)
        self.W = self.add_weight(name="W_{:s}".format(self.name),
                                 shape=(input_shape[-1], self.attention_size),
                                 initializer="glorot_normal",
                                 trainable=True)
        self.b = self.add_weight(name="b_{:s}".format(self.name),
                                 shape=(input_shape[1], 1),
                                 initializer="zeros",
                                 trainable=True)
        self.u = self.add_weight(name="u_{:s}".format(self.name),
                                 shape=(self.attention_size, 1),
                                 initializer="glorot_normal",
                                 trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x, mask=None):
        # input: (BATCH_SIZE, MAX_TIMESTEPS, EMBED_SIZE)
        # et: (BATCH_SIZE, MAX_TIMESTEPS, ATTENTION_SIZE)
        et = K.tanh(K.dot(x, self.W) + self.b)
        # at: (BATCH_SIZE, MAX_TIMESTEPS)
        at = K.softmax(K.squeeze(K.dot(et, self.u), axis=-1))
        if mask is not None:
            at *= K.cast(mask, K.floatx())
        # ot: (BATCH_SIZE, MAX_TIMESTEPS, EMBED_SIZE)
        atx = K.expand_dims(at, axis=-1)
        ot = atx * x
        # output: (BATCH_SIZE, EMBED_SIZE)
        output = K.sum(ot, axis=1)
        return output

    def compute_mask(self, input, input_mask=None):
        return None

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [13]:
from tensorflow.keras.layers import Layer, Bidirectional, LSTM, Input, Dense
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K



input_layer = Input(shape=(200, 50))

# 创建一个Bidirectional LSTM层
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(input_layer)

# 创建一个注意力层
attention = Attention(5)(lstm_layer)

# 在注意力层之后添加其他层，例如全连接层
output_layer = tf.keras.layers.Dense(5, activation="softmax")(attention)

# 创建模型
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [14]:
#model = BiLSTMModel()
# Take a look of the model layers summarry
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 200, 50)]         0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 200, 128)          58880     
 onal)                                                           
                                                                 
 attention_1 (Attention)     (None, 128)               845       
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 60370 (235.82 KB)
Trainable params: 60370 (235.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
class BiLSTMModel:
    def __init__(self):
        self.model = Sequential()
        self.model.add(Input(shape=(200, 50)))
        
        self.model.add(Bidirectional(LSTM(64, input_shape=(200, 50))))

        self.model.add(Dropout(0.2))
        
        self.model.add(Dense(5, activation='softmax'))
        self.model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
 
    def fit(self, X, Y, epochs, batch_size,verbose):
        self.model.fit(X, Y, epochs=epochs, batch_size=batch_size,verbose=verbose)
 
    def evaluate(self, X, Y, batch_size,verbose):
        return self.model.evaluate(X, Y, batch_size=batch_size,verbose=verbose)
 
    def predict(self, X):
        return self.model.predict(X)

In [15]:
# fit the model on the input and target data
model.fit(X_train,Y_train, epochs=13, batch_size=16,verbose=1)

Epoch 1/13
267/267 [==============================] - 11s 35ms/step - loss: 1.4898 - accuracy: 0.3404
Epoch 2/13
267/267 [==============================] - 9s 34ms/step - loss: 1.2137 - accuracy: 0.5094
Epoch 3/13
267/267 [==============================] - 9s 34ms/step - loss: 1.0881 - accuracy: 0.5732
Epoch 4/13
267/267 [==============================] - 10s 38ms/step - loss: 0.9773 - accuracy: 0.6246
Epoch 5/13
267/267 [==============================] - 9s 35ms/step - loss: 0.8926 - accuracy: 0.6582
Epoch 6/13
267/267 [==============================] - 9s 35ms/step - loss: 0.8208 - accuracy: 0.6901
Epoch 7/13
267/267 [==============================] - 9s 35ms/step - loss: 0.7680 - accuracy: 0.7099
Epoch 8/13
267/267 [==============================] - 9s 35ms/step - loss: 0.7116 - accuracy: 0.7331
Epoch 9/13
267/267 [==============================] - 9s 34ms/step - loss: 0.6676 - accuracy: 0.7474
Epoch 10/13
267/267 [==============================] - 9s 34ms/step - loss: 0.6141 - accu

In [16]:
#now we can check the performance of the model with the testing sets
loss,accuracy = model.evaluate(X_test, Y_test, 16,verbose = 0)
print("Loss:", loss)
print("Accuracy:", accuracy)

Loss: 1.0856750011444092
Accuracy: 0.6422535181045532


In [12]:
# Define the number of training epochs and batch size
epochs = 15
batch_size = 16

# Create empty lists to store training and test accuracy
train_accuracy_history = []
test_accuracy_history = []

for epoch in range(epochs):
    # Train the model for one epoch
    history = model.fit(X_train,Y_train, epochs=1, batch_size=16,verbose=0)
    # Calculate training accuracy
    train_accuracy = history.history['accuracy'][0]
    train_accuracy_history.append(train_accuracy)

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
    test_accuracy_history.append(test_accuracy)

    print(f'Epoch {epoch + 1}/{epochs}')
    print(f'Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}')


Epoch 1/15
Train Accuracy: 0.3390, Test Accuracy: 0.3315
Epoch 2/15
Train Accuracy: 0.4770, Test Accuracy: 0.4939
Epoch 3/15
Train Accuracy: 0.5308, Test Accuracy: 0.5502
Epoch 4/15
Train Accuracy: 0.5897, Test Accuracy: 0.5850
Epoch 5/15
Train Accuracy: 0.6174, Test Accuracy: 0.5991
Epoch 6/15
Train Accuracy: 0.6516, Test Accuracy: 0.6254
Epoch 7/15
Train Accuracy: 0.6721, Test Accuracy: 0.6376
Epoch 8/15
Train Accuracy: 0.6862, Test Accuracy: 0.6254
Epoch 9/15
Train Accuracy: 0.7094, Test Accuracy: 0.6347
Epoch 10/15
Train Accuracy: 0.7209, Test Accuracy: 0.6235
Epoch 11/15
Train Accuracy: 0.7418, Test Accuracy: 0.6357
Epoch 12/15
Train Accuracy: 0.7549, Test Accuracy: 0.6197
Epoch 13/15
Train Accuracy: 0.7669, Test Accuracy: 0.6413
Epoch 14/15
Train Accuracy: 0.7944, Test Accuracy: 0.6263
Epoch 15/15
Train Accuracy: 0.8038, Test Accuracy: 0.6263


In [18]:
Y_test

,emotion_anger,emotion_fear,emotion_joy,emotion_sadness,emotion_shame
1144,True,False,False,False,False
7106,False,False,False,False,True
296,True,False,False,False,False
3,False,False,False,True,False
1313,False,False,False,False,True
...,...,...,...,...,...
6022,False,False,False,False,True
2962,False,False,False,False,True
3721,False,True,False,False,False
1035,False,True,False,False,False


In [17]:
# Prediction using models
predictions = model.predict(X_test)

# Initialize a dictionary to store the number of correct predictions and the total number of correct predictions for each sentiment classification
accuracy_dict = {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0, 0], 4: [0, 0]}


predicted_labels = np.argmax(predictions, axis=1)

true_labels = []
for index, row in Y_test.iterrows():
    true_label_index = np.argmax(row.to_numpy())  # Index for finding the sentiment classification with the highest probability
    true_labels.append(true_label_index)

true_labels = np.array(true_labels)

# Iterate over test data and labels to calculate accuracy
for i in range(len(X_test)):
    true_label = true_labels [i]
    predicted_label = predicted_labels[i]
    accuracy_dict[true_label][1] += 1  
    if true_label == predicted_label:
        accuracy_dict[true_label][0] += 1  

# Calculate accuracy for each sentiment classification and print the results
for emotion, [correct, total] in accuracy_dict.items():
    if total > 0:
        accuracy = correct / total
        print(f"Accuracy (Emotion {emotion}): {accuracy:.2f}")
    else:
        print(f"No test data for Emotion {emotion}")

34/34 [==============================] - 1s 16ms/step
Accuracy (Emotion 0): 0.56
Accuracy (Emotion 1): 0.70
Accuracy (Emotion 2): 0.67
Accuracy (Emotion 3): 0.62
Accuracy (Emotion 4): 0.65


In [22]:
# make it a DataFrame
merged_train_accuracy_history = df = pd.DataFrame({
    'train_accuracy_history': train_accuracy_history,
    'test_accuracy_history': test_accuracy_history
})


# Export data to CSV file
file_name = f"S_accuracy_epochs_{epochs}_batch_{batch_size}.csv"
df.to_csv(file_name, index=False)

In [19]:
#Uncomment this piece of code if you want to save the model 

model.save('1trained_model_Sikai_20231020_1.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model
    
#returns a compiled model
#identical to the previous one
#model = load_model('')

e:\python3.9\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
